# LazyLM Core

> Core LazyLM classes, types, and functions

In [1]:
#| default_exp core

In [ ]:
#| hide
from nbdev.showdoc import *

In [2]:
#| hide
#| export
from dotenv import load_dotenv
from anthropic import AnthropicVertex
from anthropic.types import (
    MessageParam,
    Message
)
from fastcore.basics import *
from fastcore.test import *
from fastcore.foundation import *
from dataclasses import (
    dataclass,
    field
)
from typing import (
    List,
    Optional
)
import os
from nbdev.showdoc import show_doc

In [4]:
#| hide
load_dotenv()

True

In [14]:
#| hide
project_id = os.getenv("PROJECT_ID")
location = os.getenv("PROJECT_LOCATION")
model = os.getenv("CLAUDE_MODEL")

(project_id, location, model)

('ce-demo-space', 'us-east5', 'claude-3-5-sonnet@20240620')

This is the state called `LazyState` that will be passed to the language model. This should essentially be able to do the following:

- Keep track of the original problem
- Keep track of all of the steps done so far
- get the current step
- add a new step
- refresh the state

In [5]:
#|export
@dataclass
class LazyState:
    problem: str
    steps: List[str] = field(default_factory=list)
    current_step: int = 0

    def __post_init__(self):
        self.steps.append(self.problem)

    def add_step(self, step: str):
        self.steps.append(step)
        self.current_step += 1

    def get_context(self) -> str:
        return f"Problem: {self.problem} \n Steps so far: {self.steps}"

    def refresh(self) -> None:
        self.current_step = 0
        self.steps = [self.problem]

In [6]:
state = LazyState(problem="What is the result of f(x) = 3x + 2 when x = 5?")
state.add_step("First, we need substitute x in the function with 5")
state

LazyState(problem='What is the result of f(x) = 3x + 2 when x = 5?', steps=['What is the result of f(x) = 3x + 2 when x = 5?', 'First, we need substitute x in the function with 5'], current_step=1)

In [7]:
state.get_context()

"Problem: What is the result of f(x) = 3x + 2 when x = 5? \n Steps so far: ['What is the result of f(x) = 3x + 2 when x = 5?', 'First, we need substitute x in the function with 5']"

To keep things general, we'll have a very simple data class called `LLM` that will give us an interface for the language model client and the model version.

In [8]:
#| export
@dataclass
class LLM:
    client: AnthropicVertex
    model: str

This package will also keep support for our internal system prompt, called `lazy_system_p`

In [9]:
#| export
lazy_system_p = """
        You are a helpful assistant that can help with math problems.
        You will be given a problem and a list of steps as context, the format will be:
                
        PROBLEM: <problem>
        STEPS: <steps>

        Your job is to complete the next step and only the next step in the problem-solving process. You should never give more than one step.
        If you evaluate that the problem is done, respond with "PROBLEM DONE"
        """

With this simple state manager, we have a way to track each step of the problem-solving process and get the current context to be used for a call to a language model.

Now, let's set up a class `LazyEvaluationClient` that will do the heavy lifting of managing the state and calling the language model.

In [11]:
#| export
class LazyEvaluationClient:
    """The Lazy Evaluation Client"""
    def __init__(self, 
                 llm: LLM, # the language model to use, see `LLM` class
                 max_tokens: int = 100, # the maximum number of tokens to generate
                 state: Optional[LazyState] = None,
                 lazy_system_p: str = lazy_system_p
                ):
        self.model = llm.model
        self.client = llm.client
        self.max_tokens = max_tokens
        self.state = state
        self.lazy_system_p = lazy_system_p
        self.question_history = []

    def initalize_problem(self, problem: str) -> None:
        self.state = LazyState(problem=problem)
    
    def get_current_step(self) -> str:
        return self.state.steps[self.state.current_step]
    
    def get_next_step(self) -> str:
        if self.state is None:
            raise ValueError("Problem is not initialized, call initalize_problem first")

        messages: List[MessageParam] = [
            {
                "role": "user",
                "content": self.state.get_context()
            }
        ]

        response: Message = self.client.messages.create(
            system=self.lazy_system_p,
            model=self.model,
            messages=messages,
            max_tokens=self.max_tokens
        )
        next_step = response.content[0].text
        if next_step is not None:
            self.state.add_step(next_step.strip())
            return next_step.strip()
        else:
            raise ValueError("No next step found") 

Here is an example of using `LLM`, `LazyState`, and `LazyEvaluationClient` together.

In [15]:
lazy_state = LazyState(problem="What is the derivative of `2x^3 + x^2 + 2x + 1`? Give me the solution step-by-step")
client = AnthropicVertex(project_id=project_id, region=location)
llm = LLM(client=client, model=model)
lazy_lm = LazyEvaluationClient(llm=llm, state=lazy_state)

Let's grab the current step

In [16]:
lazy_lm.get_current_step()

'What is the derivative of `2x^3 + x^2 + 2x + 1`? Give me the solution step-by-step'

Let's get the next step

In [17]:
lazy_lm.get_next_step()

"To find the derivative of the given function, we'll use the power rule and the constant rule of differentiation. Let's start with the first term:\n\nThe derivative of 2x^3 is:\n3 * 2x^(3-1) = 6x^2"

and the next...

In [18]:
lazy_lm.get_next_step()

"Let's proceed with the next step in finding the derivative:\n\nThe derivative of x^2 is:\n2 * x^(2-1) = 2x"

Let's look at our current state

In [21]:
for step in lazy_lm.state.steps:
    print("-"*10)
    print(step)

----------
What is the derivative of `2x^3 + x^2 + 2x + 1`? Give me the solution step-by-step
----------
To find the derivative of the given function, we'll use the power rule and the constant rule of differentiation. Let's start with the first term:

The derivative of 2x^3 is:
3 * 2x^(3-1) = 6x^2
----------
Let's proceed with the next step in finding the derivative:

The derivative of x^2 is:
2 * x^(2-1) = 2x


We need a way to take the current reasoning step and query it without having the model advance to the next step in the problem-solving process.

In [32]:
#| export
@patch
def ask_question(self:LazyEvaluationClient, question:str) -> Message:
    """
    Allows the user to ask a question about the current step without affecting the model's ability to generate the next step.
    
    Args:
    question (str): The question the user wants to ask about the current step.
    
    Returns:
    str: The model's response to the question.
    """

    current_state = f"""
        System: {self.lazy_system_p}
        Problem: {self.state.problem}\n
        Context: {self.state.get_context()}
        Current step: {self.state.steps[self.state.current_step]}
    """
    prompt = f"""
        Question History: {self.question_history}
        Question: {question}\n
        Please answer the question without advancing to the next step.
        If you are asked to provide an example for a specific step, please provide an example that is not in the current context.
    """

    messages: List[MessageParam] = [
        {
            "role": "user",
            "content": prompt
        }
    ]

    response: Message = self.client.messages.create(
            system=current_state,
            model=self.model,
            messages=messages,
            max_tokens=self.max_tokens
        )
    self.question_history.append(question)
    self.question_history.append(response.content[0].text.strip())
    
    return response.content[0].text.strip()

In [27]:
lazy_lm.get_current_step()

"Let's proceed with the next step in finding the derivative:\n\nThe derivative of x^2 is:\n2 * x^(2-1) = 2x"

In [28]:
lazy_lm.ask_question("This does not make sense to me")

"I apologize for the confusion. Let me explain the current step without advancing further:\n\nThe step we're looking at is finding the derivative of the x^2 term in the original expression 2x^3 + x^2 + 2x + 1.\n\nTo find the derivative of x^2, we use the power rule of differentiation. The power rule states that for a term ax^n, the derivative is n * ax^(n-1"

## The Lazy Evaluation Flow 

This simple framework effectivly shows how we can wrape a language model capable of step-by-step reasoning to create a lazy evaluator.

This approach follows these system design steps:

- Problem Initialization: A state manager is initalized with a problem

- Prompting Strategy: Prompt the language model to generate the next step given the context in the state manager.

- State Update: State Manager records the newly generated step and updates.

- User Interaction: User interaction is held within a different state manager `question_history` which does not affect the overall state of the current problem.

- Adaptive Response: Based on the current input, the Lazy Evaluator decides to either 1) Generate the next step or 2) Provide a response to the user's question given the current state of the problem and the question history.

Finally, let's patch different model provider's APIs

In [34]:
#| export
@patch
def lazy(self: AnthropicVertex, problem: str) -> LazyEvaluationClient:
    """
    Entry point of the LazyLM Framework for the `AnthropicVertex` client API
    """
    state = LazyState(problem=problem)
    llm = LLM(client=self, model="claude-3-5-sonnet@20240620")
    return LazyEvaluationClient(llm=llm, state=state)

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()